In [1]:
import pandas as pd 
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px

data = pd.read_csv("data/heuristic_performance_final_scratch.csv")
data.set_index("instance", inplace=True)
data

,elshafei_constructive_greedy_local_search_objective,constructive_greedy_local_search_objective,5_multistart_adjacent_swap_optimal_neighbour_objective,5_multistart_adjacent_swap_first_improvement_objective,5_dissimilarity_local_search_adjacent_swap_objective,5_multistart_total_swap_optimal_neighbour_objective,5_multistart_total_swap_first_improvement_objective,5_dissimilarity_local_search_total_swap_objective,10_multistart_adjacent_swap_optimal_neighbour_objective,10_multistart_adjacent_swap_first_improvement_objective,...,10_dissimilarity_local_search_total_swap_objective,25_multistart_adjacent_swap_optimal_neighbour_objective,25_multistart_adjacent_swap_first_improvement_objective,25_dissimilarity_local_search_adjacent_swap_objective,25_multistart_total_swap_optimal_neighbour_objective,25_multistart_total_swap_first_improvement_objective,25_dissimilarity_local_search_total_swap_objective,grasp_local_search,grasp_simulated_annealing,genetic_algorithm
instance,,,,,,,,,,,,,,,,,,,,,
tai256c,0.003676,0.004289,0.168613,0.168613,0.168613,0.003465,0.003465,0.003465,0.158550,0.158550,...,0.002624,0.166798,0.166798,0.166798,0.003272,0.003272,0.003272,0.002989,0.158515,0.126427
nug16b,0.027419,0.054839,0.148387,0.148387,0.148387,0.019355,0.019355,0.019355,0.122581,0.122581,...,0.054839,0.096774,0.096774,0.096774,0.020968,0.020968,0.020968,0.009677,0.298387,0.146774
tai35b,0.019450,0.063860,0.324254,0.324254,0.324254,0.093879,0.093879,0.093879,0.519933,0.519933,...,0.015643,0.443636,0.443636,0.443636,0.016018,0.016018,0.016018,0.005118,0.579194,0.340723
chr22a,0.033788,0.061404,0.434048,0.434048,0.434048,0.091618,0.091618,0.091618,0.306368,0.306368,...,0.051657,0.351852,0.351852,0.351852,0.096491,0.096491,0.096491,0.058155,1.068876,0.427225
esc16h,0.000000,0.000000,0.130522,0.130522,0.130522,0.000000,0.000000,0.000000,0.186747,0.186747,...,0.000000,0.124498,0.124498,0.124498,0.000000,0.000000,0.000000,0.000000,0.174699,0.014056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rou12,0.056927,0.072017,0.165840,0.165840,0.165840,0.051832,0.051832,0.051832,0.075303,0.075303,...,0.001376,0.070607,0.070607,0.070607,0.000000,0.000000,0.000000,0.011065,0.193208,0.088686
esc16d,0.000000,0.000000,0.500000,0.500000,0.500000,0.000000,0.000000,0.000000,0.500000,0.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.500000,0.375000
tai20b,0.010278,0.299035,0.783443,0.783443,0.783443,0.016162,0.016162,0.016162,0.674223,0.674223,...,0.009204,0.179894,0.179894,0.179894,0.000000,0.000000,0.000000,0.003971,0.852005,0.141792


## What makes a particular instance bad across all algos

In [2]:
kmeans = KMeans(n_clusters=2, n_init=100).fit(data)
print(f"The number of instances in group 1 are {sum(kmeans.labels_)}")
print(f"The number of instances in group 0 are {len(kmeans.labels_) - sum(kmeans.labels_)}")

The number of instances in group 1 are 10
The number of instances in group 0 are 118


In [3]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(data)

In [4]:
pca_df = pd.DataFrame(principalComponents).set_index(data.index)
# pca_kmeans = KMeans(n_clusters=2, n_init=100).fit(pca_df)

pca_df['label'] = kmeans.labels_.astype(str)

data['label'] = kmeans.labels_.astype(str)

In [5]:
px.scatter(pca_df, x = 0, y=1, color='label')

In [6]:
pca_df[pca_df[1] > 4].index[0]

'esc32e'

In [7]:
data[data.index ==  pca_df[pca_df[1] > 4].index[0]]

,elshafei_constructive_greedy_local_search_objective,constructive_greedy_local_search_objective,5_multistart_adjacent_swap_optimal_neighbour_objective,5_multistart_adjacent_swap_first_improvement_objective,5_dissimilarity_local_search_adjacent_swap_objective,5_multistart_total_swap_optimal_neighbour_objective,5_multistart_total_swap_first_improvement_objective,5_dissimilarity_local_search_total_swap_objective,10_multistart_adjacent_swap_optimal_neighbour_objective,10_multistart_adjacent_swap_first_improvement_objective,...,25_multistart_adjacent_swap_optimal_neighbour_objective,25_multistart_adjacent_swap_first_improvement_objective,25_dissimilarity_local_search_adjacent_swap_objective,25_multistart_total_swap_optimal_neighbour_objective,25_multistart_total_swap_first_improvement_objective,25_dissimilarity_local_search_total_swap_objective,grasp_local_search,grasp_simulated_annealing,genetic_algorithm,label
instance,,,,,,,,,,,,,,,,,,,,,
esc32e,0.0,0.0,7.0,7.0,7.0,0.0,0.0,0.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,1


In [8]:
px.box(data, x="label", y="grasp_local_search")

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

train, test = train_test_split(data, test_size=0.4)
x_train, y_train = train.loc[:,train.columns != 'label'], train['label']
x_test, y_test = test.loc[:,test.columns != 'label'], test['label']

weights = {'0': 1, '1': 10}
logistic_model = LogisticRegression(class_weight=weights).fit(x_train,y_train)

y_pred = logistic_model.predict(x_test)

In [10]:
n_false_negatives = sum(y_pred[y_test.values == '1'] == '0')
n_false_positives = sum(y_pred[y_test.values == '0'] == '1')

print(f"number of false negatives predicted: {n_false_negatives} out of {len(y_pred)} predictions")
print(f"number of false positives predicted: {n_false_positives} out of {len(y_pred)} predictions")
print(f"average prediction accuracy of {np.mean(y_test.values == y_pred)}")

number of false negatives predicted: 0 out of 52 predictions
number of false positives predicted: 1 out of 52 predictions
average prediction accuracy of 0.9807692307692307


In [23]:
from instance_space_analysis.feature_computations import get_instance_features

column_names = ['fuck', 'problem_size', 'flow_sparsity', 'distance_sparsity', 'flow_asymmetry', 'distance_asymmetry', 'flow_dominance', 'distance_dominance', 'flow_max', 'distance_max', 'flow_min', 'distance_min', 'flow_mean', 'distance_mean', 'flow_median', 'distance_median']
x = get_instance_features(data, 'data/qapdata/')
y = pd.DataFrame(x).transpose()
y.rename({str(i): column_names[i] for i in range(16)}, axis=0)

{i: column_names[i] for i in range(16)}

y[0]

/Users/ntarvand/Documents/unimelb/qap-heuristic/instance_space_analysis/feature_computations.py:26: RuntimeWarning:

invalid value encountered in scalar divide



tai256c    256.0
nug16b      16.0
tai35b      35.0
chr22a      22.0
esc16h      16.0
           ...  
rou12       12.0
esc16d      16.0
tai20b      20.0
lipa80a     80.0
esc16e      16.0
Name: 0, Length: 128, dtype: float64

## What makes an instance hard for each algo?

## What is the best algorithm for a particular instance?